In [97]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [98]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%


clothes = ["Sushi Shirt", None]
clothes_files =  ["shirt-sushi", None]
clothes_weights = [10, 90]

faces = ["Anrgy", "Pensive", "Sad", "Smile", "Smile with dimple"]
faces_files =  ["angry", "pensive", "sad", "smile", "smile-dimple"]
faces_weights = [23.75, 23.75, 23.75, 23.75, 5]

cap = ["Ethereum Cap", "Market Cap", "Ohm Cap", "Plain Cap"]
cap_files = ["cap-eth", "cap-mkt", "cap-ohm", "cap-plain"]
cap_weights = [25, 40, 10, 15]

gender = ["Girl", None]
gender_files = ["girl", None]
gender_weights = [10, 90]

shoes = ["Barefoot", "Nikes", "Nikes with Unisocks", "Plain Sneakers"]
shoes_files = ["barefoot", "nike", "nike-socks", "plain"] 
shoes_weights = [30, 10, 1, 59]

attribute_mapping = {
    "Clothes": (clothes, clothes_weights),
    "Face": (faces, faces_weights),
    "Gender": (gender, gender_weights),
    "Head": (head, head_weights),
    "Shoes": (shoes, shoes_weights)
}

_name_filename = [{x[0]: x[1] for x in zipped} for zipped in [
        zip(clothes, clothes_files), zip(faces, faces_files), zip(cap, cap_files), 
        zip(gender, gender_files), zip(shoes, shoes_files)]
    ]
name_filename = {}
for x in _name_filename:
    name_filename.update(x)

print(name_filename)

{'Sushi Shirt': 'shirt-sushi', None: None, 'Anrgy': 'angry', 'Pensive': 'pensive', 'Sad': 'sad', 'Smile': 'smile', 'Smile with dimple': 'smile-dimple', 'Ethereum Cap': 'cap-eth', 'Market Cap': 'cap-mkt', 'Ohm Cap': 'cap-ohm', 'Plain Cap': 'cap-plain', 'Girl': 'girl', 'Barefoot': 'barefoot', 'Nikes': 'nike', 'Nikes with Unisocks': 'nike-socks', 'Plain Sneakers': 'plain'}


In [99]:
## Generate Traits

TOTAL_IMAGES = 250 # Number of random unique images we want to generate

all_images = [] 

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} #

    for attribute in attribute_mapping:
        new_image[attribute] = random.choices(*attribute_mapping[attribute])[0]
    
    
    keys = list(new_image.keys())
    for key in keys:
        if not new_image[key]:
            del new_image[key]
    
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 

    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image)

In [100]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [101]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [102]:
print(all_images)

[{'Face': 'Pensive', 'Head': 'Ohm Cap', 'Shoes': 'Nikes', 'tokenId': 0}, {'Face': 'Pensive', 'Head': 'Market Cap', 'Shoes': 'Plain Sneakers', 'tokenId': 1}, {'Face': 'Pensive', 'Head': 'Plain Cap', 'Shoes': 'Plain Sneakers', 'tokenId': 2}, {'Face': 'Anrgy', 'Gender': 'Girl', 'Head': 'Girl', 'Shoes': 'Plain Sneakers', 'tokenId': 3}, {'Face': 'Sad', 'Head': 'Market Cap', 'Shoes': 'Plain Sneakers', 'tokenId': 4}, {'Face': 'Smile', 'Head': 'Ohm Cap', 'Shoes': 'Plain Sneakers', 'tokenId': 5}, {'Face': 'Anrgy', 'Head': 'Ethereum Cap', 'Shoes': 'Barefoot', 'tokenId': 6}, {'Face': 'Anrgy', 'Head': 'Ethereum Cap', 'Shoes': 'Plain Sneakers', 'tokenId': 7}, {'Face': 'Sad', 'Head': 'Ohm Cap', 'Shoes': 'Plain Sneakers', 'tokenId': 8}, {'Face': 'Smile', 'Head': 'Ethereum Cap', 'Shoes': 'Plain Sneakers', 'tokenId': 9}, {'Face': 'Smile', 'Head': 'Plain Cap', 'Shoes': 'Barefoot', 'tokenId': 10}, {'Face': 'Sad', 'Head': 'Ethereum Cap', 'Shoes': 'Barefoot', 'tokenId': 11}, {'Face': 'Sad', 'Head': 'Plain 

In [103]:
# Get Trait Counts

trait_counts = {}

for image in all_images:
    for attribute in image:
        if attribute == "tokenId":
            continue
        if not attribute in trait_counts:
            trait_counts[attribute] = {}
        if not image[attribute] in trait_counts[attribute]:
            trait_counts[attribute][image[attribute]] = 0
        
        trait_counts[attribute][image[attribute]] += 1
            
print(trait_counts)

{'Face': {'Pensive': 51, 'Anrgy': 55, 'Sad': 56, 'Smile': 54, 'Smile with dimple': 34}, 'Head': {'Ohm Cap': 44, 'Market Cap': 59, 'Plain Cap': 49, 'Girl': 46, 'Ethereum Cap': 52}, 'Shoes': {'Nikes': 56, 'Plain Sneakers': 86, 'Barefoot': 81, 'Nikes with Unisocks': 27}, 'Gender': {'Girl': 89}, 'Clothes': {'Sushi Shirt': 95}}


In [104]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [105]:
trait_dirs = {
    "Clothes": "clothes",
    "Head": "head",
    "Shoes": "shoes",
    "Face": "faces",
    "Gender": "head"
}  
    
#### Generate Images    
for item in all_images:

    base = Image.open(f"./trait-layers/base.png")
    
    layers = [base]
    for attribute in item:
        if not attribute in trait_dirs:
            continue
        img = Image.open(f'./trait-layers/{trait_dirs[attribute]}/{name_filename[item[attribute]]}.png').convert('RGBA')
        layers.append(img)
    
    com = None
    for i in range(1, len(layers)):
        if not com:
            com = Image.alpha_composite(layers[i-1], layers[i])
        else:
            com = Image.alpha_composite(com, layers[i])
            
    #Convert to RGB
    rgb_im = com.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)

In [106]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }

for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    for attribute in attribute_mapping:
        if attribute in i:
            token["attributes"].append(getAttribute(attribute, i[attribute]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()